In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(eval_metric='mlogloss')
model.fit(X_train, y_train)

with open('xgboost_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("模型已保存为 xgboost_model.pkl")

模型已保存为 xgboost_model.pkl


In [4]:
import streamlit as st  # 导入 Streamlit 库，用于创建 Web 应用
import pandas as pd  # 导入 Pandas 库，用于数据处理
import pickle  # 导入 pickle 库，用于加载已训练的模型
import os  # 导入 os 库，用于处理文件路径
import shap  # 导入 SHAP 库，用于解释模型

# 加载模型
# 获取当前文件的目录
current_dir = os.getcwd()
print(f"当前工作目录是：{current_dir}")
# 组合当前目录与模型文件名，生成模型的完整路径
model_path = os.path.join(current_dir, 'xgboost_model.pkl')
# 打开并加载模型
with open(model_path, 'rb') as file:
    model = pickle.load(file)  # 使用 pickle 加载模型文件

# 设置 Streamlit 应用的标题
st.title("XGBoost 模型预测")

# 在侧边栏中输入特征
st.sidebar.header("输入特征")  # 侧边栏的标题
# 使用滑动条接收花萼长度，设置范围为 0.0 到 10.0，默认值为 5.0
sepal_length = st.sidebar.slider("花萼长度 (cm)", min_value=0.0, max_value=10.0, value=5.0, step=0.1)
# 使用滑动条接收花萼宽度，设置范围为 0.0 到 10.0，默认值为 5.0
sepal_width = st.sidebar.slider("花萼宽度 (cm)", min_value=0.0, max_value=10.0, value=5.0, step=0.1)
# 使用滑动条接收花瓣长度，设置范围为 0.0 到 10.0，默认值为 5.0
petal_length = st.sidebar.slider("花瓣长度 (cm)", min_value=0.0, max_value=10.0, value=5.0, step=0.1)
# 使用滑动条接收花瓣宽度，设置范围为 0.0 到 10.0，默认值为 5.0
petal_width = st.sidebar.slider("花瓣宽度 (cm)", min_value=0.0, max_value=10.0, value=5.0, step=0.1)

# 创建输入数据框，将输入的特征整理为 DataFrame 格式
input_data = pd.DataFrame({
    'sepal length (cm)': [sepal_length],
    'sepal width (cm)': [sepal_width],
    'petal length (cm)': [petal_length],
    'petal width (cm)': [petal_width]
})

# 添加预测按钮，用户点击后进行模型预测
if st.button("预测"):
    prediction = model.predict(input_data) # 使用加载的模型进行预测
    st.write(f"预测结果: {prediction[0]}")

    # 计算 SHAP 值
    explainer = shap.Explainer(model)  # 或者使用 shap.TreeExplainer(model) 来计算树模型的 SHAP 值
    shap_values = explainer(input_data)

    # 提取单个样本的 SHAP 值和期望值
    sample_shap_values = shap_values[0]  # 提取第一个样本的 SHAP 值
    expected_value = explainer.expected_value[0]  # 获取对应输出的期望值

    # 创建 Explanation 对象
    explanation = shap.Explanation(
        values=sample_shap_values[:, 0],  # 选择特定输出的 SHAP 值
        base_values=expected_value,
        data=input_data.iloc[0].values,
        feature_names=input_data.columns.tolist()
    )

    # 保存为HTML文件
    shap.save_html("shap_force_plot.html", shap.plots.force(explanation, show=False))

    # 在 Streamlit 中显示 HTML
    st.subheader("模型预测的力图")
    with open("shap_force_plot.html") as f:
        st.components.v1.html(f.read(), height=600)
        




2025-08-08 14:49:45.929 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.973 
  command:

    streamlit run C:\ANACONDA\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-08 14:49:45.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-08 14:49:45.976 Thread 'MainThread': missing ScriptR

当前工作目录是：C:\Users\19104
